# To do:
* G_prev_year seems to not be working for Mahomes
# Notes:
* Team variable tends to be good for every pos except for TE
* Still cant outperform ADP to predict points, and this is troubling, because ADP isnt even linear

# Import packages and define functions

In [1]:
###################### Import Packages #############################################
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import copy
from sklearn.ensemble import RandomForestRegressor
from skgarden import RandomForestQuantileRegressor
from sklearn.model_selection import train_test_split
from ffb_functions import *
import statsmodels.formula.api as smf
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 500)


####################################### Functions ######################################
shifted = pd.read_csv('../database_building/final_frame_teams.csv')
shifted = shifted.replace('NULL', np.nan)
# for undrafted free agents
shifted['Pick'] = shifted['Pick'].replace(0, 256)
################ Seperate database by position ####################################################
pos_dict = {}

future_features = ['delta_ppg', 'ppg_next_year', 'g_next_year']
ranking_features = ['Rk', 'OvRank']
ffb_features = ['DKPt', 'FDPt', 'VBD', 'FantPt']
categoricals = ['FantPos', 'College', 'Old_Team']
correlated_features = ['TD.3', 'RuTD', 'PaTD', 'ReTD', 'PPR', 'PaAtt', 'RuAtt'
                        , 'Rec', 'PaYds' , 'ReYds', 'RuYds', 'Int', 'Cmp', 'Tgt'
                        , 'ppg_this_year', '2PP', 'GS', 'season_frac_1']

for pos in ['TE', 'WR', 'RB', 'QB']:
    pos_dict[pos] = shifted.loc[shifted.FantPos == pos].reset_index(drop = True)
    
    # drop irrelevant variables, or variables that duplicate information
    pos_dict[pos] = pos_dict[pos].drop(future_features + categoricals + ffb_features + correlated_features
                                       , axis = 1)
    if pos != 'QB':
        pos_dict[pos] = pos_dict[pos].drop([ 'PaYds_per_PaAtt', 'PaAtt_per_game'
                                          , 'PaTD_per_PaAtt', 'Cmp_per_PaAtt', 'Int_per_PaAtt'
                                           ], axis = 1)
    else:
        pos_dict[pos] = pos_dict[pos].drop(['Rec_per_tgt', 'ReTD_per_rec'
                                           ,'Tgt_per_game', 'ReYds/R'], axis = 1)

        


/Users/JoshCoven/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
pos_dict['RB'].loc[pos_dict['RB'].Year == 2018].sort_values('Overall')#[['Name', 'PPR', 'Overall', 'pts_next_year']]
#len(pos_dict['RB'].columns)

,Rk,Name,Age,G,RuY/A,ReYds/R,Fmb,FL,2PM,PosRank,OvRank,Year,pts_next_year,G_prev_year,G_prev_year_prev_year,season_frac_2,season_frac_3,RuTD_per_Att,Rec_per_tgt,ReTD_per_rec,start_frac,Tgt_per_game,RuAtt_per_game,Fant_Share,Tm,Overall,Tm_change_flag,opp_difference,ru_opp,Pick,height,Wt,Dash,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle,Win_PCT
409,2.0,Saquon Barkley,21.0,16.0,5.01,7.92,0.0,0.0,1.0,2.0,2.0,2018.0,244.1,13.421445,13.567422,0.829184,0.555084,0.042146,0.752066,0.043956,1.000000,7.562500,16.312500,0.273249,NYG,1.45,0.0,11.0,1.0,2.0,72.000000,233.000000,4.400000,41.000000,29.000000,120.009063,6.99012,4.240000,31.2
470,3.0,Christian McCaffrey,22.0,16.0,5.01,8.10,4.0,1.0,0.0,3.0,3.0,2018.0,471.2,16.000000,13.567422,1.000000,0.555084,0.031963,0.862903,0.056075,1.000000,7.750000,13.687500,0.260508,CAR,2.65,0.0,-18.0,-112.0,8.0,71.000000,202.000000,4.480000,37.500000,10.000000,121.000000,6.57000,4.220000,43.8
113,4.0,Alvin Kamara,23.0,15.0,4.55,8.75,1.0,0.0,3.0,4.0,4.0,2018.0,248.5,16.000000,13.567422,0.968750,0.555084,0.072165,0.771429,0.049383,0.866667,7.000000,12.933333,0.218993,NOR,2.81,0.0,-54.0,-2.0,67.0,70.000000,214.000000,4.560000,39.500000,15.000000,131.000000,6.99012,4.250717,81.2
103,7.0,Ezekiel Elliott,23.0,15.0,4.72,7.36,6.0,1.0,0.0,5.0,7.0,2018.0,311.7,10.000000,15.000000,0.781250,0.625000,0.019737,0.810526,0.038961,1.000000,6.333333,20.266667,0.285405,DAL,5.26,0.0,-49.0,-2.0,4.0,72.000000,225.000000,4.470000,32.500000,20.539474,118.000000,6.99012,4.250717,62.5
280,26.0,David Johnson,27.0,16.0,3.64,8.92,3.0,2.0,1.0,10.0,26.0,2018.0,141.5,16.000000,16.000000,1.000000,0.666667,0.027132,0.657895,0.060000,1.000000,4.750000,16.125000,0.268979,ARI,10.28,0.0,-4.0,-97.0,86.0,73.000000,224.000000,4.500000,41.500000,25.000000,127.000000,6.82000,4.270000,18.8
441,14.0,James Conner,23.0,13.0,4.53,9.04,4.0,2.0,2.0,7.0,14.0,2018.0,145.5,14.000000,13.567422,0.843750,0.555084,0.055814,0.774648,0.018182,0.923077,5.461538,16.538462,0.164832,PIT,10.98,0.0,207.0,0.0,105.0,73.000000,233.000000,4.650000,29.000000,20.000000,113.000000,6.99012,4.250717,60.0
465,1.0,Todd Gurley,24.0,14.0,4.89,9.83,1.0,1.0,3.0,1.0,1.0,2018.0,219.4,15.000000,16.000000,0.906250,0.625000,0.066406,0.728395,0.067797,1.000000,5.785714,18.285714,0.227501,LAR,14.84,0.0,0.0,0.0,10.0,73.000000,222.000000,4.520000,34.634393,17.000000,120.009063,6.99012,4.250717,81.2
180,110.0,Dalvin Cook,23.0,11.0,4.62,7.63,2.0,2.0,0.0,31.0,999.0,2018.0,292.4,4.000000,13.567422,0.468750,0.555084,0.015038,0.816327,0.050000,0.909091,4.454545,12.090909,0.106049,MIN,17.18,0.0,26.0,140.0,41.0,70.000000,210.000000,4.490000,30.500000,22.000000,116.000000,7.27000,4.530000,53.3
427,40.0,Nick Chubb,23.0,16.0,5.19,7.45,0.0,0.0,0.0,15.0,40.0,2018.0,255.2,13.421445,13.567422,0.829184,0.555084,0.041667,0.689655,0.100000,0.562500,1.812500,12.000000,0.142158,CLE,17.26,0.0,-62.0,-140.0,35.0,71.000000,227.000000,4.520000,38.500000,29.000000,128.000000,7.09000,4.250000,46.7
11,24.0,Joe Mixon,22.0,14.0,4.93,6.88,0.0,0.0,0.0,9.0,24.0,2018.0,225.4,14.000000,13.567422,0.875000,0.555084,0.033755,0.781818,0.023256,0.928571,3.928571,16.928571,0.191352,CIN,19.54,0.0,0.0,0.0,48.0,70.572127,215.254279,4.520222,34.634393,20.539474,120.009063,6.99012,4.250717,37.5


# Random forest quantile regressor

In [2]:
def rfqrr_model(pos_dict, predict_year, sz, max_feats, m_depth):
    model_dict = {}
    predict_dict = {}
    outcomes = {}
    for pos in pos_dict:
        target = copy.deepcopy(pos_dict[pos])
        
        # team dummy variables. tends to be bad for QBs
        if pos != 'QB':
            dum = pd.get_dummies(target.Tm)
            target = target.drop('Tm', axis = 1)
            target = pd.concat([target, dum], axis=1)
        else:
            target = target.drop('Tm', axis = 1)
        
        # save these values to evaluate predictions later
        outcomes[pos] = (target
                         .loc[target.Year == predict_year]
                         .reset_index(drop = True)
                         [['Name', 'pts_next_year']])
        
        # set aside data to use for model prediction when the model is done
        predict_dict[pos] = (target
                             .loc[target.Year == predict_year]
                             .reset_index(drop = True)\
                             .drop(['Year', 'pts_next_year', 'Name'], axis=1)
                            #[['Overall']] #test to see if only on ADP is better than lin reg
                            ) 
        
        # make sure new values arent used in the modeling
        # only use 'sz' years of data before prediction year
        target = (target
                  .loc[(target.Year < predict_year)
                      & (target.Year > predict_year - sz)]
                  .reset_index(drop = True)
                  .drop(['Year', 'Name'], axis=1)
                 #[['Overall', 'pts_next_year']] # test to see if only ADP is better than lin reg
                 ) 

        # separate labels, targets, features
        labels = np.array(target['pts_next_year'])
        target = target.drop(['pts_next_year'], axis = 1)
        features = np.array(target)
        feature_list = list(target.columns)
        
        # run model 
        rfqr = RandomForestQuantileRegressor(random_state=0, n_estimators=3000
                                             , max_depth = m_depth
                                             #, max_features= max_feats
                                            )
        rfqr.fit(features, labels)
        model_dict[pos] = rfqr

        upper = np.concatenate(([], rfqr.predict(predict_dict[pos], quantile=75)))
        lower = np.concatenate(([], rfqr.predict(predict_dict[pos], quantile=25)))
        median = np.concatenate(([], rfqr.predict(predict_dict[pos], quantile=50)))
        
        print(pos)
        score = rfqr.score(predict_dict[pos], outcomes[pos]['pts_next_year'])
        print(rfqr.score(predict_dict[pos], outcomes[pos]['pts_next_year']))
        # Get numerical feature importances
        importances = list(rfqr.feature_importances_)
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
        [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances if pair[1] > 0]
        print('\n')
    ## get ouptuts
    final_dict = copy.deepcopy(predict_dict)
    for pos in predict_dict:
        model = model_dict[pos]
        final_dict[pos]['prediction_median'] = model.predict(predict_dict[pos], quantile=50)
        final_dict[pos]['prediction_upper'] = model.predict(predict_dict[pos], quantile=75)
        final_dict[pos]['prediction_lower'] = model.predict(predict_dict[pos], quantile=25)
        final_dict[pos]['Names'] = outcomes[pos]['Name']
        final_dict[pos]['pts_next_year'] = outcomes[pos]['pts_next_year']
        final_dict[pos]['ADP_Pos_Rank'] = final_dict[pos]['Overall'].rank()
        final_dict[pos]['Predicted_Pos_Rank'] = final_dict[pos]['prediction_median'].rank(ascending = False)
        final_dict[pos]['Actual_Next_Pos_Rank'] = final_dict[pos]['pts_next_year'].rank(ascending = False)
    return final_dict, score


def smaller_model(frame, pos, predict_year, sz):
    target = copy.deepcopy(frame)

    # team dummy variables. tends to be bad for QBs
    if pos != 'QB':
        dum = pd.get_dummies(target.Tm)
        target = target.drop('Tm', axis = 1)
        target = pd.concat([target, dum], axis=1)
    else:
        target = target.drop('Tm', axis = 1)

    # make sure new values arent used in the modeling
    # only use 'sz' years of data before prediction year
    target = (target
              .loc[(target.Year < predict_year)
                  & (target.Year > predict_year - sz)]
              .reset_index(drop = True)
              .drop(['Year', 'Name'], axis=1)
             #[['Overall', 'pts_next_year']] # test to see if only ADP is better than lin reg
             ) 

    # separate labels, targets, features
    labels = np.array(target['pts_next_year'])
    target = target.drop(['pts_next_year'], axis = 1)
    features = np.array(target)
    feature_list = list(target.columns)

    # run model 
    rfqr = RandomForestQuantileRegressor()
    
    rf_random = RandomizedSearchCV(estimator = rfqr, param_distributions = random_grid
                                   , n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    rf_random.fit(features, labels)
    
    
    return rf_random.best_params_


def one_pos_model(frame, pos, predict_year, sz, max_feats, m_depth, n_est, m_leaf):

    target = copy.deepcopy(frame)

    # team dummy variables. tends to be bad for QBs
    if 'Tm' in target.columns:
        if pos != 'QB':
            dum = pd.get_dummies(target.Tm)
            target = target.drop('Tm', axis = 1)
            target = pd.concat([target, dum], axis=1)
        else:
            target = target.drop('Tm', axis = 1)

    # save these values to evaluate predictions later
    outcomes = (target
                     .loc[target.Year == predict_year]
                     .reset_index(drop = True)
                     [['Name', 'pts_next_year']])

    # set aside data to use for model prediction when the model is done
    predict_dict = (target
                         .loc[target.Year == predict_year]
                         .reset_index(drop = True)\
                         .drop(['Year', 'pts_next_year', 'Name'], axis=1)
                        #[['Overall']] #test to see if only on ADP is better than lin reg
                        ) 

    # make sure new values arent used in the modeling
    # only use 'sz' years of data before prediction year
    target = (target
              .loc[(target.Year < predict_year)
                  & (target.Year > predict_year - sz)]
              .reset_index(drop = True)
              .drop(['Year', 'Name'], axis=1)
             #[['Overall', 'pts_next_year']] # test to see if only ADP is better than lin reg
             ) 

    # separate labels, targets, features
    labels = np.array(target['pts_next_year'])
    target = target.drop(['pts_next_year'], axis = 1)
    features = np.array(target)

    # run model 
    rfqr = RandomForestQuantileRegressor(random_state=0
                                         , n_estimators=n_est
                                         , max_depth = m_depth
                                         , max_features= max_feats
                                         , min_samples_leaf = m_leaf
                                        )
    rfqr.fit(features, labels)
    #score = rfqr.score(predict_dict, outcomes['pts_next_year'])
    score = r2_score(outcomes['pts_next_year']
                     , rfqr.predict(predict_dict)
                     # weight it so that the better someone's adp is, the more their prediction matters
                     , sample_weight = predict_dict['Overall'].max() - predict_dict['Overall'])
    
    final_dict = copy.deepcopy(predict_dict)
    final_dict['prediction_median'] = rfqr.predict(predict_dict, quantile=50)
    final_dict['Names'] = outcomes['Name']
    final_dict['pts_next_year'] = outcomes['pts_next_year']
    
    return final_dict, score

In [3]:
## try min samples split, 2, 5, 10


cross_frame = pd.DataFrame(columns = ['pos','max_features', 'max_depth', 'n_estimators', 'score'])
formula = "pts_next_year ~  prediction_median"
### window size
sz = 8
## chose the year one before, ex: want to predict for 2019 season choose 2018, last year of data
for predict_year in [
                 2014, 2015, 2016, 2017, 2018
                ]:

    for pos in ['RB', 'WR', 'TE', 'QB'
               ]:
        for max_f in ['auto'
                     ]:
            for m_dep in [15
                         ]:
                for n_est in [2000]:
                    for m_leaf in [5]:
                        final_dict, score = one_pos_model(pos_dict[pos], pos
                                                          , predict_year, sz
                                                          , max_f, m_dep, n_est
                                                         , m_leaf)
                        ## score for cross_val
                        cross_frame.loc[len(cross_frame), 'pos'] = pos
                        cross_frame.loc[len(cross_frame) - 1, 'year'] = predict_year
                        cross_frame.loc[len(cross_frame)-1, 'max_features'] = max_f
                        cross_frame.loc[len(cross_frame)-1, 'max_depth'] = int(m_dep)
                        cross_frame.loc[len(cross_frame)-1, 'n_estimators'] = int(n_est)
                        cross_frame.loc[len(cross_frame)-1, 'min_leaf_size'] = int(m_leaf)
                        cross_frame.loc[len(cross_frame) - 1, 'score'] = float(score)
                        
                        #outputs for simulation
                        output_dict = copy.deepcopy(final_dict)
                        test1 = (output_dict
                                 .sort_values('prediction_median', ascending = False)
                                    [['Names', 'prediction_median'
                                   , 'Overall', 'pts_next_year']]
                                .reset_index(drop = True))

                        test1.to_csv('simulation_{}_{}.csv'.format(pos, predict_year), index = False)
                        
            
cross_frame

## .3833
## .3631
## .55366

,pos,max_features,max_depth,n_estimators,score,year,min_leaf_size
0,RB,auto,15.0,2000.0,-0.367879,2014.0,5.0
1,WR,auto,15.0,2000.0,0.405887,2014.0,5.0
2,TE,auto,15.0,2000.0,0.135199,2014.0,5.0
3,QB,auto,15.0,2000.0,-0.203704,2014.0,5.0
4,RB,auto,15.0,2000.0,-0.0186167,2015.0,5.0
5,WR,auto,15.0,2000.0,0.159736,2015.0,5.0
6,TE,auto,15.0,2000.0,0.279317,2015.0,5.0
7,QB,auto,15.0,2000.0,0.279776,2015.0,5.0
8,RB,auto,15.0,2000.0,0.320608,2016.0,5.0
9,WR,auto,15.0,2000.0,0.177227,2016.0,5.0


In [4]:
cross_frame
test1

,Names,prediction_median,Overall,pts_next_year
0,Deshaun Watson,299.544689,34.81,321.0
1,Cam Newton,281.843509,84.74,16.7
2,Tom Brady,278.710801,104.35,263.7
3,Dak Prescott,277.665417,105.56,337.8
4,Aaron Rodgers,277.408799,44.58,278.4
5,Ben Roethlisberger,271.122817,97.84,12.7
6,Kirk Cousins,270.808078,118.36,244.4
7,Russell Wilson,269.967788,72.27,328.6
8,Andrew Luck,268.378345,60.31,0.0
9,Jared Goff,268.353501,85.95,247.5


In [103]:
cross_frame['score'] = cross_frame['score'].astype(float)
(cross_frame
 .loc[cross_frame.n_estimators == 2000]
# .groupby(['pos', 'max_depth']).mean()
)
#final_dict

#cross_frame#.dtypes

# for running backs, depth of 7, min leaf of 2

,pos,max_features,max_depth,n_estimators,score,year,min_leaf_size
0,RB,30.0,5.0,2000.0,0.363224,2016.0,5.0
1,RB,30.0,15.0,2000.0,0.363405,2016.0,5.0
2,RB,30.0,25.0,2000.0,0.363405,2016.0,5.0
3,RB,30.0,35.0,2000.0,0.363405,2016.0,5.0
4,RB,30.0,5.0,2000.0,0.290991,2017.0,5.0
5,RB,30.0,15.0,2000.0,0.293257,2017.0,5.0
6,RB,30.0,25.0,2000.0,0.293257,2017.0,5.0
7,RB,30.0,35.0,2000.0,0.293257,2017.0,5.0
8,RB,30.0,5.0,2000.0,0.408468,2018.0,5.0
9,RB,30.0,15.0,2000.0,0.412782,2018.0,5.0


In [15]:
pos_dict[pos].columns
feat = ['Overall']
added_features = ['Age', 'RuAtt_per_game'
    , 'G', 'GS', 'RuY/A', 'ReYds/R', 'Fmb', 'FL', '2PM',
       'PosRank', 'G_prev_year',
       'G_prev_year_prev_year', 'ppg_this_year', 'season_frac_1',
       'season_frac_2', 'season_frac_3', 'RuTD_per_Att', 'Rec_per_tgt',
       'ReTD_per_rec', 'start_frac', 'Tgt_per_game',
       'Fant_Share', 'Tm', 'Tm_change_flag', 'opp_difference',
       'ru_opp', 'Pick', 'height', 'Wt', 'Dash', 'Vertical', 'Bench',
       'Broad_Jump', 'Three_Cone', 'Shuttle', 'Win_PCT'
           ]

In [7]:
import random



In [21]:
## try min samples split, 2, 5, 10


cross_frame = pd.DataFrame(columns = ['pos','max_features', 'max_depth', 'n_estimators', 'score'
                                     ])
formula = "pts_next_year ~  prediction_median"
formula_adp = "pts_next_year ~  Overall"
### window size

## chose the year one before, ex: want to predict for 2019 season choose 2018, last year of data
for i in range(1, 5):
    # pick 3 features randomly from list to test with adp
    features2 = random.sample(added_features, 10)
    
    for predict_year in [2016, 2017, 2018]:
        for m_dep in [3, 7]:            
            # run model on those features
            final_dict, score = one_pos_model(pos_dict['RB'][['Name', 'pts_next_year', 'Year']
                                                             + feat + features2]
                                              , 'RB'
                                              , predict_year, 8
                                              , 'auto', m_dep, 2000
                                             , 2)
            ## score for cross_val
            cross_frame.loc[len(cross_frame), 'pos'] = pos
            cross_frame.loc[len(cross_frame) - 1, 'year'] = predict_year
            cross_frame.loc[len(cross_frame)-1, 'max_features'] = int(30)
            cross_frame.loc[len(cross_frame)-1, 'max_depth'] = int(m_dep)
            cross_frame.loc[len(cross_frame)-1, 'n_estimators'] = int(2000)
            cross_frame.loc[len(cross_frame)-1, 'min_leaf_size'] = int(2)
            cross_frame.loc[len(cross_frame)-1, 'non_adp_features1'] = features2[0]
            cross_frame.loc[len(cross_frame)-1, 'non_adp_features2'] = features2[1]
            cross_frame.loc[len(cross_frame)-1, 'non_adp_features3'] = features2[2]
            cross_frame.loc[len(cross_frame)-1, 'non_adp_features4'] = features2[3]
            cross_frame.loc[len(cross_frame)-1, 'non_adp_features5'] = features2[4]
            cross_frame.loc[len(cross_frame) - 1, 'score'] = float(score)
            # for regr
            model = smf.ols(formula, final_dict)
            cross_frame.loc[len(cross_frame) - 1, 'rsq'] = float(model.fit().rsquared)
            model2 = smf.ols(formula_adp, final_dict)
            cross_frame.loc[len(cross_frame) - 1, 'adp_rsq'] = float(model2.fit().rsquared)
                   
            
cross_frame

## previous best was for 2018, 7 depth, 2 leaf size, rsq score of around .566 as opposed to adp of .637
## on all the features

,pos,max_features,max_depth,n_estimators,score,year,min_leaf_size,non_adp_features1,non_adp_features2,non_adp_features3,non_adp_features4,non_adp_features5,rsq,adp_rsq
0,RB,30.0,3.0,2000.0,0.365412,2016.0,2.0,Fant_Share,Rec_per_tgt,Age,Tgt_per_game,Tm,0.352125,0.358560
1,RB,30.0,7.0,2000.0,0.339358,2016.0,2.0,Fant_Share,Rec_per_tgt,Age,Tgt_per_game,Tm,0.351151,0.358560
2,RB,30.0,3.0,2000.0,0.421952,2017.0,2.0,Fant_Share,Rec_per_tgt,Age,Tgt_per_game,Tm,0.458150,0.393263
3,RB,30.0,7.0,2000.0,0.419644,2017.0,2.0,Fant_Share,Rec_per_tgt,Age,Tgt_per_game,Tm,0.412260,0.393263
4,RB,30.0,3.0,2000.0,0.552854,2018.0,2.0,Fant_Share,Rec_per_tgt,Age,Tgt_per_game,Tm,0.593689,0.636918
5,RB,30.0,7.0,2000.0,0.514836,2018.0,2.0,Fant_Share,Rec_per_tgt,Age,Tgt_per_game,Tm,0.487103,0.636918
6,RB,30.0,3.0,2000.0,0.348753,2016.0,2.0,Win_PCT,RuTD_per_Att,RuAtt_per_game,season_frac_2,RuY/A,0.343970,0.358560
7,RB,30.0,7.0,2000.0,0.332342,2016.0,2.0,Win_PCT,RuTD_per_Att,RuAtt_per_game,season_frac_2,RuY/A,0.341452,0.358560
8,RB,30.0,3.0,2000.0,0.384295,2017.0,2.0,Win_PCT,RuTD_per_Att,RuAtt_per_game,season_frac_2,RuY/A,0.416141,0.393263
9,RB,30.0,7.0,2000.0,0.386326,2017.0,2.0,Win_PCT,RuTD_per_Att,RuAtt_per_game,season_frac_2,RuY/A,0.373944,0.393263


In [8]:
output_dict = copy.deepcopy(final_dict)

test1 = (output_dict
 .sort_values('prediction_median', ascending = False)
 [['Names', 'prediction_median'
   , 'Overall', 'pts_next_year']]
.reset_index(drop = True))

#test1['window'] = test1['prediction_upper'] - test1['prediction_lower']
test1.to_csv('simulation_rb.csv', index = False)

,Names,prediction_median,Overall,pts_next_year
0,Ezekiel Elliott,272.765637,5.26,311.7
1,Todd Gurley,249.626795,14.84,219.4
2,Dalvin Cook,243.165018,17.18,292.4
3,Christian McCaffrey,239.258467,2.65,471.2
4,Alvin Kamara,232.698812,2.81,248.5
5,Leonard Fournette,230.937796,26.50,259.4
6,David Johnson,230.621816,10.28,141.5
7,James Conner,224.143844,10.98,145.5
8,Joe Mixon,214.324432,19.54,225.4
9,Saquon Barkley,212.451049,1.45,244.1


# Is the model better than ADP?

In [40]:
for pos in ['RB']:
    print(pos)
    formula = "pts_next_year ~  Overall"
    model = smf.ols(formula, pos_dict['RB'].loc[pos_dict['RB'].Year < 2019])
    print(model.fit().summary())
    print(model.fit().rsquared)
    print('\n')

    
#r2_score(model.predict(pos_dict['RB'].loc[pos_dict['RB'].Year == 2018])
 #           , [pos_dict['RB'].loc[pos_dict['RB'].Year == 2018].pts_next_year])

RB
                            OLS Regression Results                            
Dep. Variable:          pts_next_year   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     262.9
Date:                Sat, 29 Feb 2020   Prob (F-statistic):           2.06e-47
Time:                        17:19:52   Log-Likelihood:                -2723.8
No. Observations:                 479   AIC:                             5452.
Df Residuals:                     477   BIC:                             5460.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    202.4460      5.983     33.839      

In [ ]:
model.predict

In [70]:
feat = ['Overall']
added_features = ['Age', 'RuAtt_per_game'
    , 'G', 'RuY/A', 'ReYds/R', 'Fmb', 'FL', '2PM',
       'PosRank', 'G_prev_year',
       'G_prev_year_prev_year',
       'season_frac_2', 'season_frac_3', 'RuTD_per_Att', 'Rec_per_tgt',
       'ReTD_per_rec', 'start_frac', 'Tgt_per_game',
       'Fant_Share', 'Tm', 'Tm_change_flag', 'opp_difference',
       'ru_opp', 'Pick', 'height', 'Wt', 'Dash', 'Vertical', 'Bench',
       'Broad_Jump', 'Three_Cone', 'Shuttle', 'Win_PCT'
           ]    
    
## try min samples split, 2, 5, 10
cross_frame = pd.DataFrame(columns = ['pos','max_features', 'max_depth', 'n_estimators', 'score'])
formula = "pts_next_year ~  prediction_median"


### window size
sz = 8
pos = 'RB'
max_f = 'auto'
m_dep = 7
n_est = 2000
m_leaf = 2

## choose the year one before, ex: want to predict for 2019 season choose 2018, last year of data
for predict_year in [#2016, 2017, 
                 2018
                ]:
    for i in feat + added_features:
        try:
            final_dict, score = one_pos_model(pos_dict[pos].drop([i], axis = 1)
                                              , pos
                                              , predict_year, sz
                                              , max_f, m_dep, n_est
                                             , m_leaf)
            ## score for cross_val
            cross_frame.loc[len(cross_frame), 'pos'] = pos
            cross_frame.loc[len(cross_frame) - 1, 'year'] = predict_year
            cross_frame.loc[len(cross_frame)-1, 'feature'] = i
            cross_frame.loc[len(cross_frame)-1, 'max_depth'] = int(m_dep)
            cross_frame.loc[len(cross_frame)-1, 'n_estimators'] = int(n_est)
            cross_frame.loc[len(cross_frame)-1, 'min_leaf_size'] = int(m_leaf)
            cross_frame.loc[len(cross_frame) - 1, 'score'] = float(score)
        except:
            pass
                   
            
cross_frame

## .3833
## .3631
## .55366

,pos,max_features,max_depth,n_estimators,score,year,feature,min_leaf_size
0,RB,NaN,7.0,2000.0,0.402933,2018.0,Overall,2.0
1,RB,NaN,7.0,2000.0,0.559509,2018.0,Age,2.0
2,RB,NaN,7.0,2000.0,0.567936,2018.0,RuAtt_per_game,2.0
3,RB,NaN,7.0,2000.0,0.562148,2018.0,G,2.0
4,RB,NaN,7.0,2000.0,0.562014,2018.0,RuY/A,2.0
5,RB,NaN,7.0,2000.0,0.563927,2018.0,ReYds/R,2.0
6,RB,NaN,7.0,2000.0,0.56084,2018.0,Fmb,2.0
7,RB,NaN,7.0,2000.0,0.56004,2018.0,FL,2.0
8,RB,NaN,7.0,2000.0,0.561814,2018.0,2PM,2.0
9,RB,NaN,7.0,2000.0,0.561385,2018.0,PosRank,2.0


In [72]:
cross_frame.sort_values('score')


,pos,max_features,max_depth,n_estimators,score,year,feature,min_leaf_size
0,RB,NaN,7.0,2000.0,0.402933,2018.0,Overall,2.0
20,RB,NaN,7.0,2000.0,0.546178,2018.0,Tm,2.0
29,RB,NaN,7.0,2000.0,0.551133,2018.0,Bench,2.0
33,RB,NaN,7.0,2000.0,0.555026,2018.0,Win_PCT,2.0
30,RB,NaN,7.0,2000.0,0.557948,2018.0,Broad_Jump,2.0
28,RB,NaN,7.0,2000.0,0.558688,2018.0,Vertical,2.0
12,RB,NaN,7.0,2000.0,0.559034,2018.0,season_frac_2,2.0
10,RB,NaN,7.0,2000.0,0.559176,2018.0,G_prev_year,2.0
1,RB,NaN,7.0,2000.0,0.559509,2018.0,Age,2.0
15,RB,NaN,7.0,2000.0,0.55987,2018.0,Rec_per_tgt,2.0


# True feature importance
* Score diff when removed